In [ ]:
df_feature = df_clean[['diagnosis','area_mean', 'concavity_mean', 'concave points_mean', 'radius_worst',
       'perimeter_worst', 'area_worst', 'concavity_worst','concave points_worst']]
df_feature.head()

In [ ]:
df_train = df_feature.head(int(0.8*len(df_feature)))
df_hold = df_feature.tail(int(0.2*len(df_feature)))

In [ ]:
sample_20 = df_train.sample(int(0.2*len(df_train)), random_state=2)
sample_40 = df_train.sample(int(0.4*len(df_train)), random_state=2)
sample_60 = df_train.sample(int(0.6*len(df_train)), random_state=2)
sample_80 = df_train.sample(int(0.8*len(df_train)), random_state=2)

In [ ]:
sm = from_pandas(df_train)
sm_20 = from_pandas(sample_20)
sm_40 = from_pandas(sample_40)
sm_60 = from_pandas(sample_60)
sm_80 = from_pandas(sample_80)

In [ ]:
sm.remove_edges_below_threshold(0.8)
viz = plot_structure(
    sm,
    graph_attributes={"scale": "2"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK)
Image(viz.draw(format='png'))

In [ ]:
def jaccard_similarity(g, h, th1, th2):
    g.remove_edges_below_threshold(th1)
    h.remove_edges_below_threshold(th2)
    a = g.edges
    b = h.edges
    i = set(a).intersection(b)
    return round(len(i) / (len(a) + len(b) - len(i)),3)